# Pima-Diabetes-Prediction

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from sklearn.model_selection import RepeatedKFold
from sklearn.cross_validation import train_test_split
from  sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import binarize

In [25]:
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
data = pd.read_csv('pima-indians-diabetes.data', names=col_names)


In [4]:
data.count()

pregnant    768
glucose     768
bp          768
skin        768
insulin     768
bmi         768
pedigree    768
age         768
label       768
dtype: int64

In [9]:
data.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
pregnant    768 non-null int64
glucose     768 non-null int64
bp          768 non-null int64
skin        768 non-null int64
insulin     768 non-null int64
bmi         768 non-null float64
pedigree    768 non-null float64
age         768 non-null int64
label       768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [11]:
data.shape

(768, 9)

In [12]:
data.describe()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [21]:
data.isnull().values.any()

False

In [26]:
def plot_correlated(data, size=11):
    """
    Function to show plot of correlation between the data columns
    data : the data from pandas Dataframe
    size : the horizontal and vertical size of the plot
    
    Result : blue - cyan - yellow - red -darkred = less to more correlated columns = 0 ---- 1 
    """
    corr = data.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)

In [23]:
data.corr()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age
pregnant,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341
glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514
bp,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528
skin,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970
insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163
bmi,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242
pedigree,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561
age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000


In [28]:
feature_columns = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age']
predicted_class = ['label']

X = data[feature_columns].values
y = data[predicted_class].values
#split_test_size =0.30

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_test_size, random_state=10)

In [29]:
label = data['label']
data.drop('label', axis=1, inplace=True)
X_p, y_p = data, label

In [30]:
# selection of relevant features
#label = data['Outcome']
#data.drop('', axis=1, inplace=True)

#X, y = data, label
#X_p = pd.read_csv("diabetes.csv", usecols = [0,1,2,3,4,5,6,7])
#y_p = pd.read_csv("diabetes.csv", usecols = ['Outcome'])


#X_p.head(5)



In [31]:
#X_train.shape

In [32]:
print(X)

[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]


In [63]:
#from sklearn.model_selection import KFold 
kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=10) 

for train_index, test_index in kf.split(X):
    #print("Train:", train_index, "Validation:",test_index)
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]

In [34]:
X_train.shape

(615, 8)

In [35]:
y_train.shape

(615, 1)

In [36]:
X_test.shape

(153, 8)

In [37]:
y_test.shape

(153, 1)

In [38]:
from sklearn.linear_model import LogisticRegression

log_reg_model = LogisticRegression(C=1.0, random_state=10)

log_reg_model.fit(X_train, y_train.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=10, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [39]:
lr_predict_train_data = log_reg_model.predict(X_train)

from sklearn import metrics

print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_train, lr_predict_train_data)))

Accuracy = 0.792


In [40]:
lr_predict_test_data = log_reg_model.predict(X_test)

from sklearn import metrics


print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, lr_predict_test_data)))

Accuracy = 0.752


In [41]:
print("Confusion matrix")

print("{0}".format(metrics.confusion_matrix(y_test, lr_predict_test_data, labels=[1,0])))

print("Classification report")

print("{0}".format(metrics.classification_report(y_test, lr_predict_test_data, labels=[1,0])))

Confusion matrix
[[34 30]
 [ 8 81]]
Classification report
             precision    recall  f1-score   support

          1       0.81      0.53      0.64        64
          0       0.73      0.91      0.81        89

avg / total       0.76      0.75      0.74       153



In [42]:
log_reg_model = LogisticRegression(C=1.0, class_weight="balanced", random_state=11)

log_reg_model.fit(X_train, y_train.ravel())

lr_predict_test_data = log_reg_model.predict(X_test)


In [43]:
print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, lr_predict_test_data)))

Accuracy = 0.791


In [44]:
print("Confusion matrix")

print("{0}".format(metrics.confusion_matrix(y_test, lr_predict_test_data, labels=[1,0])))

print("Classification report")

print("{0}".format(metrics.classification_report(y_test, lr_predict_test_data, labels=[1,0])))


Confusion matrix
[[54 10]
 [22 67]]
Classification report
             precision    recall  f1-score   support

          1       0.71      0.84      0.77        64
          0       0.87      0.75      0.81        89

avg / total       0.80      0.79      0.79       153



In [45]:
from sklearn.linear_model import LogisticRegressionCV
lrcv_model = LogisticRegressionCV(n_jobs=-1, random_state=10, Cs=10, cv=5, refit=True, class_weight="balanced")
lrcv_model.fit(X, y.ravel())

LogisticRegressionCV(Cs=10, class_weight='balanced', cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=-1, penalty='l2', random_state=10,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

In [46]:
lrcv_predict_test_data = lrcv_model.predict(X_test)

In [47]:
print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, lrcv_predict_test_data)))

Accuracy = 0.797


In [48]:
print("Confusion matrix")

print("{0}".format(metrics.confusion_matrix(y_test, lrcv_predict_test_data, labels=[1,0])))

print("Classification report")

print("{0}".format(metrics.classification_report(y_test, lrcv_predict_test_data, labels=[1,0])))


Confusion matrix
[[52 12]
 [19 70]]
Classification report
             precision    recall  f1-score   support

          1       0.73      0.81      0.77        64
          0       0.85      0.79      0.82        89

avg / total       0.80      0.80      0.80       153



In [51]:
from sklearn.ensemble  import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import binarize
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [53]:
%%time
# GridSearch procedure = cpu demanding
gradient_boost_eval = GradientBoostingClassifier(random_state=0)

params = {
    'learning_rate': [0.05, 0.1, 0.5],
    'max_features': [0.5, 1],
    'max_depth': [3, 4, 5] 
         }

grid_search = GridSearchCV(gradient_boost_eval, params, cv=10, scoring='roc_auc')
grid_search.fit(X_p,y_p)

print(grid_search.best_params_)
print(grid_search.best_score_)

{'learning_rate': 0.05, 'max_depth': 3, 'max_features': 0.5}
0.833971391263058
CPU times: user 1min 35s, sys: 21.4 ms, total: 1min 35s
Wall time: 1min 35s


In [54]:
# GBC belong to the Random Tree Classifier family
# a complex model who perform well on small Dataset
# We set our hyper-params with the best params providing by GridSearch
gradient_boost = GradientBoostingClassifier(learning_rate=0.05, max_depth=3, max_features=0.5, random_state=0)


In [55]:
#Cross Validation handle the .fit and .predict method
cross_val_score(gradient_boost, X_p, y_p, cv=10, scoring='roc_auc').mean()

0.833980056980057

In [62]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
#from sklearn.model_selection import KFold 
kf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=10) 

for train_index, test_index in kf.split(X_p):
    #print("Train:", train_index, "Validation:",test_index)
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]

In [57]:
#Adjusting the Threshold for improvement

In [58]:
gradient_boost_imp = GradientBoostingClassifier(learning_rate=0.05, max_depth=3, max_features=0.5, random_state=0)



gradient_boost_imp.fit(X_train, y_train)

/home/praveen/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=3,
              max_features=0.5, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=0, subsample=1.0, verbose=0,
              warm_start=False)

In [59]:
# Storing the predicted probabilities of classification
y_pred_prob = gradient_boost_imp.predict_proba(X_test)[:, 1]

In [61]:
# level of predictive precision
roc_auc_score(y_test, y_pred_prob)

0.8600852272727273